In [1]:
import pandas as pd
from collections import Counter
import requests
import re
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

## Check what are the most popular format files in the resources csv

In [2]:
resources_df = pd.read_csv("../input_files/resources.csv", sep=";")

In [3]:
freqs_formats = sorted(Counter(resources_df.format).items(), key=lambda x:x[1], reverse=True)
freqs_formats

[('json', 41212),
 ('shp', 32148),
 ('csv', 27717),
 ('zip', 25047),
 ('document', 19763),
 ('xml', 9957),
 (nan, 8148),
 ('html', 7518),
 ('xls', 5557),
 ('pdf', 4476),
 ('image', 3437),
 ('ods', 2006),
 ('xlsx', 1703),
 ('.asc, .las, .glz', 1048),
 ('geojson', 787),
 ('kml', 764),
 ('bin', 760),
 ('kmz', 515),
 ('txt', 495),
 ('api', 391),
 ('dbf', 352),
 ('web page', 302),
 ('ecw', 297),
 ('mif', 292),
 ('esri rest', 278),
 ('odata', 259),
 ('doc', 248),
 ('shapefile', 201),
 ('wfs', 162),
 ('odt', 151),
 ('docx', 148),
 ('dxf', 144),
 ('png', 133),
 ('csv/utf8', 130),
 ('dwg', 130),
 ('data', 122),
 ('shape', 95),
 ('tiff', 83),
 ('7z', 82),
 ('gtfs', 68),
 ('jpg', 63),
 ('ksh', 57),
 ('get', 56),
 ('url', 56),
 ('wms', 55),
 ('0.csv', 50),
 ('rar', 49),
 ('rtf', 43),
 ('csv, json, excel, shp', 42),
 ('gpkg', 40),
 ('text/turtle', 39),
 ('02.2019.01-00.xml', 38),
 ('autre', 37),
 ('obj', 36),
 ('shp (wgs84)', 35),
 ('excel', 34),
 ('gpx', 33),
 ('csv, api', 32),
 ('application/vnd.

Things to notice:
1. Among the 8148 nan's, can we infer the format in another way?
2. What's up with the "asc, las, glz" and document formats ?
3. The distribution is heavily left sided with a long tail of very infrequent formats. From which threshold can we begin ignoring formats and still do not affect too much the relative counts (percentages) ? 
4. What is inside the zip files ?

## Question 3:

In [15]:
less_500 = sum(f[1] for f in freqs_formats if f[1] < 500)
less_500
print("Total percentage of resources with format frequency lower than 500: {}".format(100*less_500/len(resources_df)))

Total percentage of resources with format frequency lower than 500: 3.6308040316688186


I guess we can ignore those formats with frequency lower than 500. That gives the following formats:

[('json', 41212),
 ('shp', 32148),
 ('csv', 27717),
 ('zip', 25047),
 ('document', 19763),
 ('xml', 9957),
 (nan, 8148),
 ('html', 7518),
 ('xls', 5557),
 ('pdf', 4476),
 ('image', 3437),
 ('ods', 2006),
 ('xlsx', 1703),
 ('.asc, .las, .glz', 1048),
 ('geojson', 787),
 ('kml', 764),
 ('bin', 760),
 ('kmz', 515)]

## Question 1:
What's up with the nans ? We can try to infer the type from the url

In [16]:
extension_re = re.compile(r'/[\w_\-]+\.(\w+)$')

def try_to_get_format(dataset):
    dataset.fillna("")
    if dataset.format != "":
        print(dataset.format)
        return dataset.format
    # Try to get format from url
    if dataset.url:
        print("Trying regex")
        matcho = extension_re.findall(dataset.url)
        if matcho:
            return matcho[-1]
    # Go to the url and try to get it from the header
        try:
            print("Trying download")
            r = requests.head(dataset.url, allow_redirects=True)
            extension = r.headers["Content-Type"].split("/")[-1]
            return extension
        except:
            return ""
    return ""
        


In [ ]:

resources_dd = dd.from_pandas(resources_df, npartitions=5)

# res = resources_dd.map_partitions(lambda df: df.apply(lambda x: try_to_get_format(x), axis=1), meta=("result", str)).compute(scheduler="processes")


[                                        ] | 0% Completed |  3.1scsv
nan
xls
csv
mif−mid
html
xml
pdf
html
csv
csv
nan
zip
nan
csv, json, excel, shp
csv
csv
csv
html
html
html
csv
json
html
csv
json
csv
csv
csv
zip
csv
html
csv
html
xml
xml
csv
csv
s57
csv
csv
html
csv
csv
ods
csv
html
txt
txt
txt
txt
html
xls
xls
html
pdf
html
xlsx
csv
csv
zip
csv
ods
html
xls
html
ods
html
xls
dbase
html
pdf
html
html
txt
zip
xlsx
nan
nan
nan
halbfischgmail.com-l26xe-156_1.tgz
csv
zip
kml
zip
csv, json, excel, shp
ecw
document
html
geojson
kml
csv
json
csv
sql
csv
csv
csv
csv
csv
csv
csv
csv
csv
json
shp
csv
json
nan
zip
xls
ods
csv
xls
ods
csv
csv
csv
csv
doc
csv
pdf
txt
xml
xls
ods
rss
csv
json
shp
csv
kml
csv
kml
csv
csv
ods
zip
pdf
bin
csv
json
csv
xlsx
xlsx
tiff
html
csv
xls
doc
doc
xlsx
csv
html
.xlsx
csv
pdf
0.csv
csv
csv
csv
csv
[                                        ] | 0% Completed |  3.2scsv
0.csv
0.csv
csv
odt
ods
csv
.xlsx
html
txt
csv
geojson
xls
txt
csv
html
csv
csv
.xlsx
txt
txt
txt

In [15]:
res

0                           csv
1                           NaN
2                           xls
3                           csv
4                       mif−mid
5                          html
6                           xml
7                           pdf
8                          html
9                           csv
10                          csv
11                          NaN
12                          zip
13                          NaN
14        csv, json, excel, shp
15                          csv
16                          csv
17                          csv
18                         html
19                         html
20                         html
21                          csv
22                         json
23                         html
24                          csv
25                         json
26                          csv
27                          csv
28                          csv
29                          zip
                  ...          
199788  

In [ ]:


resources_df["maybe_format"] = resources_df.apply(try_to_get_format, axis=1)


Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Tryin

Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Tryin

Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying downlo

Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying downlo

Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying reg

...

Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Try

Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Tr

Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download


Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying download
Trying regex
Trying

Trying regex
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Trying regex
Trying download
Tr

In [122]:
Counter(resources_df.maybe_format)
# foo = resources_df.loc[1]


Counter({'csv': 27717,
         nan: 8148,
         'xls': 5557,
         'mif−mid': 2,
         'html': 7518,
         'xml': 9957,
         'pdf': 4476,
         'zip': 25047,
         'csv, json, excel, shp': 42,
         'json': 41212,
         's57': 1,
         'ods': 2006,
         'txt': 495,
         'xlsx': 1703,
         'dbase': 8,
         'halbfischgmail.com-l26xe-156_1.tgz': 1,
         'kml': 764,
         'ecw': 297,
         'document': 19763,
         'geojson': 787,
         'sql': 20,
         'shp': 32148,
         'doc': 248,
         'rss': 13,
         'bin': 760,
         'tiff': 83,
         '.xlsx': 8,
         '0.csv': 50,
         'odt': 151,
         'excel': 34,
         'shape': 95,
         'csv/utf8': 130,
         'tar.gz': 19,
         'kmz': 515,
         'neptune': 19,
         'gtfs': 68,
         'api': 391,
         '.asc, .las, .glz': 1048,
         'asc': 2,
         'shp (l93)': 5,
         'autre': 37,
         'csv, api': 32,
         'jpg

In [119]:
foo.fillna("")



Unnamed: 0                                                      1
_id                          1a7c68bf-6730-4a32-8858-bff8666a6deb
checksum                                                         
created_at                                2014-05-07 04:16:00.245
dataset.id                               53699760a3a729239d204c97
description                   Plan TCL édition Février 2014. ©TCL
extras          {'check:available': True, 'check:count-availab...
filesize                                                         
filetype                                                   remote
format                                                           
metrics         {'nb_uniq_visitors': 0, 'nb_visits': 0, 'nb_hi...
mime                                                             
modified                                  2014-04-18 08:03:22.584
published                                 2014-09-04 06:45:43.692
title                                                    Plan TCL
type      

In [3]:
a = "goo.csv"
a[-3:]

'csv'

In [12]:
ll = [(1,2),(1,2),(1,2)]
list(zip(*ll))[:1][0]

(1, 1, 1)